In [1]:
import pandas as pd
import numpy as np
import sympy as sp
import seaborn as sns
import matplotlib.pyplot as plt


from scipy.optimize import minimize
from scipy.optimize import differential_evolution

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')


In [2]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(object)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01_00_20' if x == '01 - 00-<=020' else
                                                          '02_20_40' if x == '02 - 20-<=040' else
                                                          '03_40_60' if x == '03 - 40-<=060' else
                                                          '04_60_80' if x == '04 - 60-<=080' else
                                                          '05_80_100' if x == '05 - 80-<=100' else
                                                          '02_20_40' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor_Mais' if x == 'Corretor Mais' else 
                                                   'Corretor_Mais' if x == 'CorretorMais'
                                                                   else    'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: 'p_1_10' if x < 10 else
                                                      'p_10_35' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x:     'Sem_Sinistro' if x == 'Sem Sinistro' else
                                                                      '01_05_anos' if x == '01 ano' else
                                                                      '01_05_anos' if x == '02 anos' else
                                                                      '01_05_anos' if x == '03 anos' else
                                                                      '01_05_anos' if x == '04 anos' else
                                                                      '01_05_anos' if x == '05 anos' else '06_inf')


    df['UF_bin'] = df['UF'].apply(lambda x:     'AL_CE_RS'  if x == 'AL' else
                                                'AL_CE_RS'  if x == 'CE' else
                                                'AL_CE_RS'  if x == 'RS' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'AP' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'DF' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'ES' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PA' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PB' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'RN' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'SC' else 
                                                'GO_MA_MG'             if x == 'GO' else 
                                                'GO_MA_MG'             if x == 'MA' else 
                                                'GO_MA_MG'             if x == 'MG' else 
                                                'MT_PR'                if x == 'MT' else
                                                'MT_PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  
                                                'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1_SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2_RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2_RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3_RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3_RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '7 - Renov BB com sinistro' else  
                                                                          '2_RenovPropria')
  
    df['MD_tipo_renov_bin'] = df['MD_tipo_renov'].apply(lambda x:          '2_RenovPropria_sem_sin'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                           '3_RenovPropria_com_sin'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                           'Error')
   
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: 'f_80_101'    if x < 101 else
                                                                   'f_101_120'   if x >= 101  else 
                                                                   'f_80_101')   
    
    return df

In [3]:
def demand_model(df):
    import pandas as pd
    import numpy as np 
    P   = df['premio']
    X2  = np.array(df['GBM_mercado'])
    X3  = np.array(df['Premio_IC_100'])
    X4  = np.array(df['premio_anterior'])
    X5  = np.array(df['MD_tempo_casa_2'])
    X6  = np.array(df['per_com'])
    X7  = np.array(df['per_com_bin_p_1_10'])
    X1  = np.array(df['is_casco_ofer'])
    X8  = np.array(df['canal_bin_Corretor_Mais'])
    X9  = np.array(df['Cod_End_bin_13509'])
    X10 = np.array(df['fator_ajuste_bin_f_101_120'])
    X11 = np.array(df['classe_bonus_bin_00'])
    X12 = np.array(df['UF_bin_AL_CE_RS'])
    X13 = np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
    X14 = np.array(df['UF_bin_GO_MA_MG'])
    X15 = np.array(df['UF_bin_MT_PR'])
    X16 = np.array(df['UF_bin_RJ'])
    X17 = np.array(df['MD_comb_bin_Gasolina'])
    X18 = np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
    X19 = np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
    X20 = np.array(df['fx_ir_12m_bin_01_00_20'])
    X21 = np.array(df['fx_ir_12m_bin_03_40_60'])
    X22 = np.array(df['fx_ir_12m_bin_04_60_80'])
    X23 = np.array(df['fx_ir_12m_bin_05_80_100'])
    X24 = np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
    X25 = np.array(df['idade_ult_RNS_bin_01_05_anos'])
    X26 = np.array(df['idade_ult_RNS_bin_06_inf'])

    def demanda(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
        D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
        return D/(1+D)


    #df['Renewal_Demand'] = demanda(P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26)
    
    return  demanda(P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26)
  

In [4]:
def relatorio(df):
    import pandas as pd
    import numpy as np 
    
    df["tarifa"] = df["premio"]
    df["per_com_pct"] = df['per_com']/100
    df["premio_prob"] = df["Renewal_Demand"]*df["premio"]
    df["tarifa_prob"] = df["retencao_otimizada"]*df["tarifa"]
    df["premio_anterior_prob"] = df["Renewal_Demand"]*df["premio_anterior"]
    df["Act_cost_prob"] = df["Renewal_Demand"]*df["Act_cost"]                                                                
    df["comissao_valor"] = df["per_com_pct"] *df["premio"]
    df["comissao_valor_prob"] = df["per_com_pct"] *df["premio_prob"]


    tarifa              = np.mean(df["tarifa"])
    tarifa_w            = np.sum(df["tarifa_prob"])/np.sum(df["retencao_otimizada"])
    apolice_ofertada    = np.round(df.shape[0],0)
    apolice_estimada    = np.round(np.sum(df["Renewal_Demand"]),0)
    premio_ofertado     = np.mean(df["premio"])
    retencao_estimada   = np.mean(df["Renewal_Demand"])
    premio_esperado     = np.sum(df["premio_prob"])/np.sum(df["Renewal_Demand"])
    risco_ofertado      = np.mean(df["Act_cost"])
    risco_esperado      = np.sum(df["Act_cost_prob"])/np.sum(df["Renewal_Demand"])
    sp_ofertado         = np.sum(df["Act_cost"])/np.sum(df["premio"])
    sp_esperado         = np.sum(df["Act_cost_prob"])/np.sum(df["premio_prob"])
    comis_ofertado      = np.sum(df["comissao_valor"])/np.sum(df["premio"])
    comis_esperado      = np.sum(df["comissao_valor_prob"])/np.sum(df["premio_prob"])
    premio_anterior     = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["Renewal_Demand"])
    ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
    ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
    premio_anterior   = np.mean(df_lote["premio_anterior"])
    premio_anterior_w   = np.sum(df_lote["premio_anterior_prob"])/np.sum(df_lote["Renewal_Demand"])

    Relative_Core_Premium_Increase_Offered = (premio_ofertado-premio_anterior)/premio_anterior
    Relative_Core_Premium_Increase_Written = (premio_anterior-premio_anterior_w)/premio_anterior_w
    Relative_Core_Optimization_Increase_Offered=(premio_ofertado-tarifa)/tarifa
    Relative_Core_Optimization_Increase_Written=(premio_anterior-premio_anterior_w)/premio_anterior_w
    Core_Premium_Increase_Offered=(premio_ofertado-premio_anterior)/apolice_estimada
    Core_Premium_Increase_Written=(premio_ofertado-premio_anterior_w)/apolice_estimada


    tabela = pd.DataFrame([apolice_ofertada,apolice_estimada,
                            retencao_estimada,
                            premio_ofertado,premio_esperado,
                            premio_anterior,premio_anterior_w,
                            risco_ofertado,risco_esperado,
                            comis_ofertado, comis_esperado,
                            sp_ofertado,sp_esperado,
                            ic_ofertado,ic_esperado,      
                            Relative_Core_Premium_Increase_Offered,
                            Relative_Core_Premium_Increase_Written,
                            Relative_Core_Optimization_Increase_Written,
                            Core_Premium_Increase_Offered,
                            Core_Premium_Increase_Written
                          ])



    tabela = tabela.rename(columns = {0 : 'Total'})

    tabela.index = ['Apolices Ofertadas:', 
                    'Apolices Renovadas:',
                    '%Retenção:',
                    'Premio Ofertado:',
                    'Premio Renovado:',  
                    'Premio Anterior Ofertado:',
                    'Premio Anterior Renovado:',
                    'Risco Ofertado', 
                    'Risco Renovado', 
                    '%Comissão Ofertada:',
                    '%Comissão Renovada:',
                    '%SP Ofertada:', 
                    '%SP Renovada:',
                    '%IC Ofertado:', 
                    '%IC Renovado:',
                    'Relative_Core_Premium_Increase_Offered:',
                    'Relative_Core_Premium_Increase_Written:',
                    'Relative_Core_Optimization_Increase_Written:',
                    'Core_Premium_Increase_Offered:',
                    'Core_Premium_Increase_Written:']
    return tabela

In [5]:
def relatorio_otim(df):
    import pandas as pd
    import numpy as np 
    
    df["tarifa"] = df["premio"]
    df["retencao_otimizada"] = df["retencao_otimizada"]
    df["precos_otimizados"] = df["precos_otimizados"]
    
    df["per_com_pct"] = df['per_com']/100
    df["premio_prob"] = df["retencao_otimizada"]*df["precos_otimizados"]
    df["premio_anterior_prob"] = df["retencao_otimizada"]*df["premio_anterior"]
    df["tarifa_prob"] = df["retencao_otimizada"]*df["tarifa"]
    df["Act_cost_prob"] = df["retencao_otimizada"]*df["Act_cost"]                                                                
    df["comissao_valor"] = df["per_com_pct"] *df["precos_otimizados"]
    df["comissao_valor_prob"] = df["per_com_pct"] *df["premio_prob"]


    tarifa              = np.mean(df["tarifa"])
    tarifa_w            = np.sum(df["tarifa_prob"])/np.sum(df["retencao_otimizada"])
    apolice_ofertada    = np.round(df.shape[0],0)
    apolice_estimada    = np.round(np.sum(df["retencao_otimizada"]),0)
    premio_ofertado     = np.mean(df["precos_otimizados"])
    retencao_estimada   = np.mean(df["retencao_otimizada"])
    premio_esperado     = np.sum(df["premio_prob"])/np.sum(df["retencao_otimizada"])
    risco_ofertado      = np.mean(df["Act_cost"])
    risco_esperado      = np.sum(df["Act_cost_prob"])/np.sum(df["retencao_otimizada"])
    sp_ofertado         = np.sum(df["Act_cost"])/np.sum(df["precos_otimizados"])
    sp_esperado         = np.sum(df["Act_cost_prob"])/np.sum(df["premio_prob"])
    comis_ofertado      = np.sum(df["comissao_valor"])/np.sum(df["precos_otimizados"])
    comis_esperado      = np.sum(df["comissao_valor_prob"])/np.sum(df["premio_prob"])
    premio_anterior     = np.mean(df["premio_anterior"])
    premio_anterior_w   = np.sum(df["premio_anterior_prob"])/np.sum(df["retencao_otimizada"])
    ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
    ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
    premio_anterior   = np.mean(df_lote["premio_anterior"])
    premio_anterior_w   = np.sum(df_lote["premio_anterior_prob"])/np.sum(df_lote["retencao_otimizada"])

    Relative_Core_Premium_Increase_Offered = (premio_ofertado-premio_anterior)/premio_anterior
    Relative_Core_Premium_Increase_Written = (premio_anterior-premio_anterior_w)/premio_anterior_w
    Relative_Core_Optimization_Increase_Offered=(premio_ofertado-tarifa)/tarifa
    Relative_Core_Optimization_Increase_Written=(premio_anterior-premio_anterior_w)/premio_anterior_w
    Core_Premium_Increase_Offered=(premio_ofertado-premio_anterior)/apolice_estimada
    Core_Premium_Increase_Written=(premio_ofertado-premio_anterior_w)/apolice_estimada

    
    
    
    tabela = pd.DataFrame([apolice_ofertada,apolice_estimada,
                            retencao_estimada,
                            premio_ofertado,premio_esperado,
                            premio_anterior,premio_anterior_w,
                            risco_ofertado,risco_esperado,
                            comis_ofertado, comis_esperado,
                            sp_ofertado,sp_esperado,
                            ic_ofertado,ic_esperado,      
                            Relative_Core_Premium_Increase_Offered,
                            Relative_Core_Premium_Increase_Written,
                            Relative_Core_Optimization_Increase_Written,
                            Core_Premium_Increase_Offered,
                            Core_Premium_Increase_Written
                            ])


    tabela = tabela.rename(columns = {0 : 'Total'})

    tabela.index = ['Apolices Ofertadas:', 
                    'Apolices Renovadas:',
                    '%Retenção:',
                    'Premio Ofertado:',
                    'Premio Renovado:',  
                    'Premio Anterior Ofertado:',
                    'Premio Anterior Renovado:',
                    'Risco Ofertado', 
                    'Risco Renovado', 
                    '%Comissão Ofertada:',
                    '%Comissão Renovada:',
                    '%SP Ofertada:', 
                    '%SP Renovada:',
                    '%IC Ofertado:', 
                    '%IC Renovado:',
                    'Relative_Core_Premium_Increase_Offered:',
                    'Relative_Core_Premium_Increase_Written:',
                    'Relative_Core_Optimization_Increase_Written:',
                    'Core_Premium_Increase_Offered:',
                    'Core_Premium_Increase_Written:']

    return tabela

In [6]:
def elasticidade(df):
    
    df['tarifa']=df['premio']
    df['numbers'] = df['premio']/df['tarifa']
    if df['numbers'].any()>1 :
        df['X27'] = df['numbers']-1
    else:
         df['X27']=1
    
    
    P=(df['premio'])
    X1=np.array(df['is_casco_ofer'])
    X2=np.array(df['GBM_mercado'])
    X3=np.array(df['Premio_IC_100'])
    X4=np.array(df['premio_anterior'])
    X5=np.array(df['MD_tempo_casa_2'])
    X6=np.array(df['per_com'])
    X7=np.array(df['per_com_bin_p_1_10'])
    X8=np.array(df['canal_bin_Corretor_Mais'])
    X9=np.array(df['Cod_End_bin_13509'])
    X10=np.array(df['fator_ajuste_bin_f_101_120'])
    X11=np.array(df['classe_bonus_bin_00'])
    X12=np.array(df['UF_bin_AL_CE_RS'])
    X13=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
    X14=np.array(df['UF_bin_GO_MA_MG'])
    X15=np.array(df['UF_bin_MT_PR'])
    X16=np.array(df['UF_bin_RJ'])
    X17=np.array(df['MD_comb_bin_Gasolina'])
    X18=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
    X19=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
    X20=np.array(df['fx_ir_12m_bin_01_00_20'])
    X21=np.array(df['fx_ir_12m_bin_03_40_60'])
    X22=np.array(df['fx_ir_12m_bin_04_60_80'])
    X23=np.array(df['fx_ir_12m_bin_05_80_100'])
    X24=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
    X25=np.array(df['idade_ult_RNS_bin_01_05_anos'])
    X26=np.array(df['idade_ult_RNS_bin_06_inf'])
    X26=np.array(df['idade_ult_RNS_bin_06_inf'])
    X27=np.array(df['X27'])

    
    
    
    # Definindo as variáveis simbólicas
    P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, X27 = sp.symbols('P X1 X2 X3 X4 X5 X6 X7 X8 X9 X10 X11 X12 X13 X14 X15 X16 X17 X18 X19 X20 X21 X22 X23 X24 X25 X26 X27')

    # Definindo a função de demanda
    demanda = sp.exp(1.3356- 0.2576 * sp.log(P)+ 1.3647 * (P / X1)- 0.8782 * sp.log(P / X2)- 0.795 * sp.log(P / X3)- 1.6731 * sp.log(P / X4)+ 0.3797 * sp.log(X5 + 1)+ 0.0011 * X6+ 0.1253 * X7+ 0.1334 * X8- 0.1382 * X9+ 0.3919 * X10- 0.1397 * X11- 0.2305 * X12- 0.2832 * X13+ 0.0024 * X14- 0.1476 * X15+ 0.2817 * X16+ 0.3336 * X17- 0.4115 * X18+ 0.0127 * X19- 0.73 * X20+ 0.4343 * X21+ 1.1066 * X22+ 1.9561 * X23- 0.0742 * X24- 0.8459 * X25- 0.7094 * X26)/(1+sp.exp(1.3356- 0.2576 * sp.log(P)+ 1.3647 * (P / X1)- 0.8782 * sp.log(P / X2)- 0.795 * sp.log(P / X3)- 1.6731 * sp.log(P / X4)+ 0.3797 * sp.log(X5 + 1)+ 0.0011 * X6+ 0.1253 * X7+ 0.1334 * X8- 0.1382 * X9+ 0.3919 * X10- 0.1397 * X11- 0.2305 * X12- 0.2832 * X13+ 0.0024 * X14- 0.1476 * X15+ 0.2817 * X16+ 0.3336 * X17- 0.4115 * X18+ 0.0127 * X19- 0.73 * X20+ 0.4343 * X21+ 1.1066 * X22+ 1.9561 * X23- 0.0742 * X24- 0.8459 * X25- 0.7094 * X26 -3*X27))

    # Derivada da demanda em relação ao preço
    demanda_derivada_P = sp.diff(demanda, P)

    # Funções lambdify para avaliação numérica
    demanda_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26,X27), demanda, 'numpy')
    demanda_derivada_P_func = sp.lambdify((P, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26,X27), demanda_derivada_P, 'numpy')


    # Supondo que df é um DataFrame do pandas com os dados necessários
    
    # Extraindo os valores das variáveis
    lista_variaveis_na_ordem = ['is_casco_ofer','GBM_mercado','Premio_IC_100','premio_anterior','MD_tempo_casa_2','per_com','per_com_bin_p_1_10','canal_bin_Corretor_Mais','Cod_End_bin_13509','fator_ajuste_bin_f_101_120','classe_bonus_bin_00','UF_bin_AL_CE_RS','UF_bin_AP_DF_ES_PA_PB_RN_SC','UF_bin_GO_MA_MG','UF_bin_MT_PR','UF_bin_RJ','MD_comb_bin_Gasolina','md_tipo_renov_ant_bin_1_SeguroNovo','md_tipo_renov_ant_bin_3_RenovCongenere','fx_ir_12m_bin_01_00_20','fx_ir_12m_bin_03_40_60','fx_ir_12m_bin_04_60_80','fx_ir_12m_bin_05_80_100','MD_tipo_renov_bin_3_RenovPropria_com_sin','idade_ult_RNS_bin_01_05_anos','idade_ult_RNS_bin_06_inf','X27']

    X_vals = np.array(df.loc[:, lista_variaveis_na_ordem])
    P_val = df['premio'].values

    # Calculando os valores de demanda e sua derivada
    D_val = demanda_func(P_val, *X_vals.T)
    dD_dP_val = demanda_derivada_P_func(P_val, *X_vals.T)

    # Calculando a elasticidade-preço da demanda
    elasticidade_preco = (dD_dP_val * P_val) / D_val
    return elasticidade_preco

## Base Para escorar e otimizar

In [7]:
df_lote_102024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/AJUSTADO Base Case Earnix - Grupo Segmentacao.csv",
                             dtype={'chave': 'string'},
                     encoding='latin-1',
                     delimiter='|')

df_lote_102024['GBM_mercado']  = df_lote_102024['GBM_mercado']/(1-df_lote_102024['per_com']/100)

In [8]:
df_lote_escorar=df_lote_102024.copy()

In [9]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [10]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [11]:
df_lote = categorizar(df_lote_escorar)

In [12]:
df_lote.shape

(1473, 144)

In [13]:
df_cat = df_lote[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_glm=pd.get_dummies(df_cat)
df_lote=pd.concat([df_lote,df_cat_glm],axis=1)

In [ ]:
#df_lote.drop(['Renewal Demand','Premium Parameter'],axis='columns', inplace=True)

In [14]:
df_lote['Renewal_Demand'] = demand_model(df_lote)

In [15]:
df_lote['elasticidade']=elasticidade(df_lote)

In [ ]:
#relatorio(df)

In [16]:
df_lote['Renewal_Demand'].mean()

0.513467580404544

In [17]:
df_lote['elasticidade'].mean()

-3.216150870569343

## Otimização Teste 

In [ ]:
#print(df_lote_otim['elasticidade'].value_counts(ascending=False))

In [ ]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra_2 == "Segmentacao"')
df_lote_otim_atu.shape

In [ ]:
df_lote_otim = df_lote_otim_atu
df_lote_otim.shape

In [ ]:
df_lote_otim['Renewal_Demand'].mean()

#########################################

In [ ]:
%%time
######
import sympy as sp
from scipy.optimize import fsolve

df = df_lote_otim


P=(df['premio'])
X1=np.array(df['is_casco_ofer'])
X2=np.array(df['GBM_mercado'])
X3=np.array(df['Premio_IC_100'])
X4=np.array(df['premio_anterior'])
X5=np.array(df['MD_tempo_casa_2'])
X6=np.array(df['per_com'])
X7=np.array(df['per_com_bin_p_1_10'])
X8=np.array(df['canal_bin_Corretor_Mais'])
X9=np.array(df['Cod_End_bin_13509'])
X10=np.array(df['fator_ajuste_bin_f_101_120'])
X11=np.array(df['classe_bonus_bin_00'])
X12=np.array(df['UF_bin_AL_CE_RS'])
X13=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X14=np.array(df['UF_bin_GO_MA_MG'])
X15=np.array(df['UF_bin_MT_PR'])
X16=np.array(df['UF_bin_RJ'])
X17=np.array(df['MD_comb_bin_Gasolina'])
X18=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
X19=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
X20=np.array(df['fx_ir_12m_bin_01_00_20'])
X21=np.array(df['fx_ir_12m_bin_03_40_60'])
X22=np.array(df['fx_ir_12m_bin_04_60_80'])
X23=np.array(df['fx_ir_12m_bin_05_80_100'])
X24=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X25=np.array(df['idade_ult_RNS_bin_01_05_anos'])
X26=np.array(df['idade_ult_RNS_bin_06_inf'])
X26=np.array(df['idade_ult_RNS_bin_06_inf'])
X27=np.array(df['X27'])


def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26):
    D=np.exp(1.3356 -0.2576 * np.log(P)  + 1.3647 * np.divide(P, X1)-0.8782 * np.log(np.divide(P, X2))-0.795 * np.log(np.divide(P, X3))-1.6731 * np.log((np.divide(P, X4)))  + 0.3797 * np.log(X5 + 1)  + 0.0011 * X6  + 0.1253 * X7  + 0.1334 * X8-0.1382 * X9  + 0.3919 * X10-0.1397 * X11-0.2305 * X12-0.2832 * X13  + 0.0024 * X14-0.1476 * X15  + 0.2817 * X16  + 0.3336 * X17-0.4115 * X18  + 0.0127 * X19-0.73 * X20  + 0.4343 * X21  + 1.1066 * X22  + 1.9561 * X23-0.0742 * X24 -0.84590*X25- 0.7094 * X26)
    return D/(1+D)


# Definir a função objetivo que maximiza a margem esperada otimizada 
def objetivo(P):
    
    df['precos']= P *1.23
    
   
    df['tarifa']=df['premio']
   
    df['numbers'] = df['precos']/df['tarifa']

    if (df['numbers']>1).any():
        df['X27'] = df['numbers']-1
    else:
         df['X27']=0
        
    # Calcular a retenção otimizada com os preços dados
    df['retencao_otimizada1'] = calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)
    
    # Calcular a margem esperada
    df['margem_esperada_otimizada'] =   df['retencao_otimizada1']*(df['precos'] - df['Act_cost'])
    
  
    margem_esperada_total = df['margem_esperada_otimizada'].sum() 

    # Maximizar a margem 
    return -margem_esperada_total

# Definir os limites para os preços otimizado
nome_variaveis = ['LB','UB']
limites=np.array(df.loc[:,nome_variaveis])

# Executar a otimização 
resultado = differential_evolution(objetivo, bounds=limites, strategy='best1bin', maxiter=3000, popsize=11,seed=df.shape[0])

# Resultados
print(resultado.message) 
print("Tamanho da Base:", df.shape[0])
precos_otimizados = resultado.x
margem_esperada_total_otimizada = -resultado.fun


In [ ]:
print(df['precos'].mean()),
print(np.mean(precos_otimizados))

In [ ]:
df['precos_otimizados'] = precos_otimizados

df['retencao_otimizada'] = calcular_retencao(df['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)

df['margem_esperada_base_case'] = df['Renewal_Demand']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()
  
    
#delta_q = (df['retencao_otimizada']-df['Renewal_Demand'])
#delta_p = (df['precos_otimizados']-df['premio'])
   
#delta_q_b = ((df['retencao_otimizada']+df['Renewal_Demand'])/2)
#delta_p_b = ((df['precos_otimizados']+df['premio'])/2)

#df["elasticidade"] = (delta_q/delta_p)*(delta_p_b/delta_q_b)
#df["elasticidade_2"] = (df['retencao_otimizada']/df['Renewal_Demand']-1)/(df['precos_otimizados']/df['premio']-1)  

#df["elasticidade"] = df["elasticidade"].fillna(-1)
#df["elasticidade_2"] = df["elasticidade_2"].fillna(-1)

df['elasticidade_bin'] = pd.cut(x=df['elasticidade'], bins=[-3.6,-3.35,-3.1,-2.85,-2.6,-2.35,-2.1,-1.85,-1.6,-1.35,-1.1,-0.85,-0.6,-0.35,0,])  

df.replace([np.inf, -np.inf], -1, inplace=True)



#df["SP_ofer_python"] = df["Act_cost"]/df["precos_otimizados"]
#df["IC_ofer_python"] = df["per_com_pct"] + df["SP_ofer_python"] +  0.2749
#df["SP_ofer_eranix"] = df["Act_cost"]/df["Premium Parameter"]
#df["IC_ofer_eranix"] = df["SP_ofer_eranix"] + df["per_com_pct"]+  0.2749
#df["varia_prem_anterior_py"] = df["precos_otimizados"]/df["premio_anterior"]
#df["varia_prem_anterior_earnix"] = df["Premium Parameter"]/df["premio_anterior"]
#df["varia_prem_ofer_py"] = df["precos_otimizados"]/df["premio"]
#df["varia_prem_ofer_earnix"] = df["Premium Parameter"]/df["premio"]
#df["varia_prem_py_earnix"] = df["precos_otimizados"]/df["Premium Parameter"]


#df['IC_ofer_python_bin'] = pd.cut(x=df['IC_ofer_python'], bins=[0.7,0.95,0.99,1,1.03,1.9])  
#df['IC_ofer_eranix'] = pd.cut(x=df['IC_ofer_eranix'], bins=[0.7,0.95,0.99,1,1.03,1.9]) 


# Exibir os resultados
print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Base Case:", df['Renewal_Demand'].mean())
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Premio Medio Otimizado:", df['precos_otimizados'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Elasticidade:", df['elasticidade'].mean())


In [ ]:
df['retencao_otimizada'].mean()

In [ ]:
df['Renewal_Demand'].mean()

In [ ]:
df['precos_otimizados'].mean()

In [ ]:
#relatorio_otim(df)

#criando a função

In [ ]:
df_relatorio_otim = df.copy()


df_relatorio_otim['precos_otimizados'] = df_relatorio_otim['precos_otimizados']


df_relatorio_otim['batente_otim']=np.round(df_relatorio_otim['precos_otimizados']/df_relatorio_otim['premio'],2)
df_relatorio_otim['batente_otim_prem_ant']=np.round(df_relatorio_otim['precos_otimizados']/df_relatorio_otim['premio_anterior'],2)



df_relatorio_otim['retencao_otimizada'] = calcular_retencao(df_relatorio_otim['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26)


df_relatorio_otim["per_com_pct"] = df_relatorio_otim['per_com']/100
df_relatorio_otim["premio_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["Act_cost_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["Act_cost"]                                                                
df_relatorio_otim["comissao_valor"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["comissao_valor_prob"] = df_relatorio_otim["per_com_pct"]*df_relatorio_otim["premio_prob"]

df_relatorio_otim["premio_anterior_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["premio_anterior"]

comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])

premio_anterior_ofer    = np.mean(df_relatorio_otim["premio_anterior"])
premio_anterior_w   = np.sum(df_relatorio_otim["premio_anterior_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])


apolice_ofertada  = np.round(df_relatorio_otim.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_otim["retencao_otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio_otim["precos_otimizados"])
retencao_estimada = np.mean(df_relatorio_otim["retencao_otimizada"])
premio_esperado   = np.sum(df_relatorio_otim["premio_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
risco_ofertado    = np.mean(df_relatorio_otim["Act_cost"])
risco_esperado    = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
sp_ofertado       = np.sum(df_relatorio_otim["Act_cost"])/np.sum(df_relatorio_otim["precos_otimizados"])
sp_esperado       = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749


print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))

print('comis_ofertado" : %s'  % np.round(comis_ofertado*100,2))
print('comis_esperado" : %s'  % np.round(comis_esperado*100,2))

print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

print('premio_anterior_ofer" : %s'  %np.round( premio_anterior_ofer,2))
print('premio_anterior_w" : %s'  %np.round( premio_anterior_w,2))

In [ ]:
df_relatorio_otim['precos_otimizados'].mean()

In [ ]:
df_relatorio_otim['batente_otim_prem_ant'].describe()

In [ ]:
df_relatorio_otim['batente_otim'].describe()

In [ ]:
df_relatorio_otim['elasticidade'].describe()

In [ ]:
analitico_202410_ajustado=df_relatorio_otim
analitico_202410_ajustado.drop(['Renewal_Demand','retencao_otimizada1'],axis='columns', inplace=True)
#analitico_202410_ajustado.to_csv(r'analitico_202410_ajustado.csv', sep='|' ,header='true')
analitico_202410_ajustado.to_csv(r'analitico_202410_ajustado_elaticidade.csv', sep='|' ,header='true')

In [ ]:
%%time
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

#df = df_lote_otim.iloc[0:100,]
df = df_lote_otim

lista_variaveis_na_ordem = [
    'is_casco_ofer',
    'GBM_mercado',
    'Premio_IC_100',
    'premio_anterior',
    'MD_tempo_casa_2',
    'per_com',
    'per_com_bin_p_1_10',
    'canal_bin_Corretor_Mais',
    'Cod_End_bin_13509',
    'fator_ajuste_bin_f_101_120',
    'classe_bonus_bin_00',
    'UF_bin_AL_CE_RS',
    'UF_bin_AP_DF_ES_PA_PB_RN_SC',
    'UF_bin_GO_MA_MG',
    'UF_bin_MT_PR',
    'UF_bin_RJ',
    'MD_comb_bin_Gasolina',
    'md_tipo_renov_ant_bin_1_SeguroNovo',
    'md_tipo_renov_ant_bin_3_RenovCongenere',
    'fx_ir_12m_bin_01_00_20',
    'fx_ir_12m_bin_03_40_60',
    'fx_ir_12m_bin_04_60_80',
    'fx_ir_12m_bin_05_80_100',
    'MD_tipo_renov_bin_3_RenovPropria_com_sin',
    'idade_ult_RNS_bin_01_05_anos',
    'idade_ult_RNS_bin_06_inf'
]

indicadora = np.array(df.loc[:, lista_variaveis_na_ordem])

precos_iniciais = np.array(df['premio']) 
target_demand = df['Renewal_Demand'].mean()
custo = df['Act_cost']

# Define a função de demanda
def demanda(X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, P):
    D = np.exp(
        1.3356 - 0.2576 * np.log(P) + 1.3647 * (P / X1) - 0.8782 * np.log(P / X2) - 0.795 * np.log(P / X3) - 1.6731 * np.log(P / X4) +
        0.3797 * np.log(X5 + 1) + 0.0011 * X6 + 0.1253 * X7 + 0.1334 * X8 - 0.1382 * X9 + 0.3919 * X10 - 0.1397 * X11 - 0.2305 * X12 -
        0.2832 * X13 + 0.0024 * X14 - 0.1476 * X15 + 0.2817 * X16 + 0.3336 * X17 - 0.4115 * X18 + 0.0127 * X19 - 0.73 * X20 +
        0.4343 * X21 + 1.1066 * X22 + 1.9561 * X23 - 0.0742 * X24 - 0.84590 * X25 - 0.7094 * X26
    )
    return D / (1 + D)

# Define a função de Lagrange
def func(X, indicadora):
    P = X[:-1]  # Extraindo o vetor de preços [p1, p2, ..., pn]
    L = X[-1]   # Extraindo o multiplicador de Lagrange
    n = len(P)
    
    
    
    
    # Função objetivo e a soma das demandas
    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(
            indicadora[i, 0], indicadora[i, 1], indicadora[i, 2], indicadora[i, 3], indicadora[i, 4], indicadora[i, 5],
            indicadora[i, 6], indicadora[i, 7], indicadora[i, 8], indicadora[i, 9], indicadora[i, 10], indicadora[i, 11],
            indicadora[i, 12], indicadora[i, 13], indicadora[i, 14], indicadora[i, 15], indicadora[i, 16], indicadora[i, 17],
            indicadora[i, 18], indicadora[i, 19], indicadora[i, 20], indicadora[i, 21], indicadora[i, 22], indicadora[i, 23],
            indicadora[i, 24], indicadora[i, 25], P[i]
        )
        objetivo += D_i * (P[i] - custo[i])
        soma_demandas += D_i
    
    return objetivo - L * (soma_demandas - target_demand * df.shape[0])

def dfunc(X, indicadora):
    dLambda = np.zeros(len(X))
    h = 1e-3  # Passo usado na diferença finita
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X + dX, indicadora) - func(X - dX, indicadora)) / (2 * h)
    return dLambda

def otimizar_precos(indicadora, precos_iniciais):
    X_inicial = np.append(precos_iniciais, 0)
    
    
    X_otimizado = fsolve(dfunc, X_inicial, args=(indicadora), xtol=1e-06, maxfev=100)
    
    precos_otimizados = X_otimizado[:-1]
    lambda_otimizado = X_otimizado[-1]
    
    return precos_otimizados, lambda_otimizado

precos_otimizados, lambda_otimizado = otimizar_precos(indicadora, precos_iniciais)

print("λ:", lambda_otimizado)
print("Preços Otimizados:", precos_otimizados)

In [ ]:
precos_otimizados.describe()

In [ ]:
df['precos_otimizados']=precos_otimizados

In [ ]:
df[['precos_otimizados','premio']].describe()

In [ ]:
X1=np.array(df['is_casco_ofer'])
X2=np.array(df['GBM_mercado'])
X3=np.array(df['Premio_IC_100'])
X4=np.array(df['premio_anterior'])
X5=np.array(df['MD_tempo_casa_2'])
X6=np.array(df['per_com'])
X7=np.array(df['per_com_bin_p_1_10'])
X8=np.array(df['canal_bin_Corretor_Mais'])
X9=np.array(df['Cod_End_bin_13509'])
X10=np.array(df['fator_ajuste_bin_f_101_120'])
X11=np.array(df['classe_bonus_bin_00'])
X12=np.array(df['UF_bin_AL_CE_RS'])
X13=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X14=np.array(df['UF_bin_GO_MA_MG'])
X15=np.array(df['UF_bin_MT_PR'])
X16=np.array(df['UF_bin_RJ'])
X17=np.array(df['MD_comb_bin_Gasolina'])
X18=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
X19=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
X20=np.array(df['fx_ir_12m_bin_01_00_20'])
X21=np.array(df['fx_ir_12m_bin_03_40_60'])
X22=np.array(df['fx_ir_12m_bin_04_60_80'])
X23=np.array(df['fx_ir_12m_bin_05_80_100'])
X24=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X25=np.array(df['idade_ult_RNS_bin_01_05_anos'])
X26=np.array(df['idade_ult_RNS_bin_06_inf'])
X26=np.array(df['idade_ult_RNS_bin_06_inf'])


np.mean(calcular_retencao(df['precos_otimizados'],X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26) )

In [ ]:
df['Renewal_Demand'].mean()

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize


df = df_lote_otim.iloc[0:100,]

lista_variaveis_na_ordem = [
    'is_casco_ofer',
    'GBM_mercado',
    'Premio_IC_100',
    'premio_anterior',
    'MD_tempo_casa_2',
    'per_com',
    'per_com_bin_p_1_10',
    'canal_bin_Corretor_Mais',
    'Cod_End_bin_13509',
    'fator_ajuste_bin_f_101_120',
    'classe_bonus_bin_00',
    'UF_bin_AL_CE_RS',
    'UF_bin_AP_DF_ES_PA_PB_RN_SC',
    'UF_bin_GO_MA_MG',
    'UF_bin_MT_PR',
    'UF_bin_RJ',
    'MD_comb_bin_Gasolina',
    'md_tipo_renov_ant_bin_1_SeguroNovo',
    'md_tipo_renov_ant_bin_3_RenovCongenere',
    'fx_ir_12m_bin_01_00_20',
    'fx_ir_12m_bin_03_40_60',
    'fx_ir_12m_bin_04_60_80',
    'fx_ir_12m_bin_05_80_100',
    'MD_tipo_renov_bin_3_RenovPropria_com_sin',
    'idade_ult_RNS_bin_01_05_anos',
    'idade_ult_RNS_bin_06_inf'
]

indicadora = np.array(df.loc[:, lista_variaveis_na_ordem])

precos_iniciais = np.array(df['premio']) * 1
target_demand = df['Renewal_Demand'].mean()
custo = df['Act_cost']

# Define a função de demanda
def demanda(X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16, X17, X18, X19, X20, X21, X22, X23, X24, X25, X26, P):
    D = np.exp(
        1.3356 - 0.2576 * np.log(P) + 1.3647 * (P / X1) - 0.8782 * np.log(P / X2) - 0.795 * np.log(P / X3) - 1.6731 * np.log(P / X4) +
        0.3797 * np.log(X5 + 1) + 0.0011 * X6 + 0.1253 * X7 + 0.1334 * X8 - 0.1382 * X9 + 0.3919 * X10 - 0.1397 * X11 - 0.2305 * X12 -
        0.2832 * X13 + 0.0024 * X14 - 0.1476 * X15 + 0.2817 * X16 + 0.3336 * X17 - 0.4115 * X18 + 0.0127 * X19 - 0.73 * X20 +
        0.4343 * X21 + 1.1066 * X22 + 1.9561 * X23 - 0.0742 * X24 - 0.84590 * X25 - 0.7094 * X26
    )
    return D / (1 + D)

# Define a função de Lagrange
def func(X, indicadora):
    P = X[:-1]  # Extraindo o vetor de preços [p1, p2, ..., pn]
    L = X[-1]   # Extraindo o multiplicador de Lagrange
    n = len(P)
    
    # Função objetivo e a soma das demandas
    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(
            indicadora[i, 0], indicadora[i, 1], indicadora[i, 2], indicadora[i, 3], indicadora[i, 4], indicadora[i, 5],
            indicadora[i, 6], indicadora[i, 7], indicadora[i, 8], indicadora[i, 9], indicadora[i, 10], indicadora[i, 11],
            indicadora[i, 12], indicadora[i, 13], indicadora[i, 14], indicadora[i, 15], indicadora[i, 16], indicadora[i, 17],
            indicadora[i, 18], indicadora[i, 19], indicadora[i, 20], indicadora[i, 21], indicadora[i, 22], indicadora[i, 23],
            indicadora[i, 24], indicadora[i, 25], P[i]
        )
        objetivo += D_i * (P[i] - custo[i])
        soma_demandas += D_i
    
    return objetivo - L * (soma_demandas - target_demand * df.shape[0])

def dfunc(X, indicadora):
    dLambda = np.zeros(len(X))
    h = 1e-3  # Passo usado na diferença finita
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X + dX, indicadora) - func(X - dX, indicadora)) / (2 * h)
    return dLambda

def otimizar_precos(indicadora, precos_iniciais):
    # Iniciando o vetor X com os preços iniciais e um valor inicial para o multiplicador de Lagrange
    X_inicial = np.append(precos_iniciais, 0)
    
    # Definindo os limites para os preços
    bounds = [(0.9 * p, 1.1 * p) for p in precos_iniciais] + [(None, None)]  # Adiciona limite para L
    
    # Função objetivo para minimizar
    def objetivo(X):
        return -func(X, indicadora)
    
    # Otimização com restrições
    resultado = minimize(objetivo, X_inicial, bounds=bounds, method='L-BFGS-B')
    
    precos_otimizados = resultado.x[:-1]
    lambda_otimizado = resultado.x[-1]
    
    return precos_otimizados, lambda_otimizado

precos_otimizados, lambda_otimizado = otimizar_precos(indicadora, precos_iniciais)

print("λ:", lambda_otimizado)

In [ ]:
print(precos_otimizados.mean()),
print(df['premio'].mean())

In [ ]:
target_demand

In [ ]:
import numpy as np
from scipy.optimize import fsolve

# Definindo a função de demanda usando Regressão Logística 
def demanda(beta, QP, P):
    return 1 / (1 + np.exp(-(beta[0] + beta[1]*QP + beta[2]*P)))

# Definindo a função de margem
def margem(QP, P, beta):
    D = demanda(beta, QP, P)
    return P * D - (10 * QP + 2 * P)  # Exemplo de função de custo

# Função objetivo com restrição usando Lagrange 
def lagrange_system(vars, beta):
    QP, P, lambda_ = vars

    # Derivadas parciais de Lagrange
    dL_dQP = -10 + lambda_ * 1
    dL_dP = -2 + lambda_ * 1
    # Restrição
    constraint = QP + P - 1000

    return [dL_dQP, dL_dP, constraint]

# Coeficientes da regressão logística (usado no exemplo da imagem)
beta = [0.5, 1.2, -0.9]

# Condições iniciais (chute inicial)
x0 = [100, 50, 1]  # QP, P, Lambda

# Resolvendo o sistema de equações de Lagrange 
solution = fsolve(lagrange_system, x0, args=(beta,))

QP_opt, P_opt, lambda_opt = solution

# Resultados
margem_max = margem(QP_opt, P_opt, beta)

print(f"QP ótimo: {QP_opt}")
print(f"P ótimo: {P_opt}")
print(f"Lambda: {lambda_opt}")
print(f"Margem máxima: {margem_max}")


In [ ]:
data = {   
    'premio': np.random.uniform(200, 350, 10),
    'is_casco_ofer': np.random.randint(0, 2, 10),
    'GBM_mercado': np.random.uniform(0.5, 1.5, 10),
    'Premio_IC_100': np.random.uniform(100, 200, 10),
    'premio_anterior': np.random.uniform(150, 300, 10),
    'MD_tempo_casa_2': np.random.uniform(1, 5, 10),
    'per_com': np.random.uniform(0.1, 1.0, 10),
    'per_com_bin_p_1_10': np.random.randint(0, 2, 10),
    'canal_bin_Corretor_Mais': np.random.randint(0, 2, 10),
    'Cod_End_bin_13509': np.random.randint(0, 2, 10),
    'fator_ajuste_bin_f_101_120': np.random.randint(0, 2, 10),
    'classe_bonus_bin_00': np.random.randint(0, 2, 10),
    'UF_bin_AL_CE_RS': np.random.randint(0, 2, 10),
    'UF_bin_AP_DF_ES_PA_PB_RN_SC': np.random.randint(0, 2, 10),
    'UF_bin_GO_MA_MG': np.random.randint(0, 2, 10),
    'UF_bin_MT_PR': np.random.randint(0, 2, 10),
    'UF_bin_RJ': np.random.randint(0, 2, 10),
    'MD_comb_bin_Gasolina': np.random.randint(0, 2, 10),
    'md_tipo_renov_ant_bin_1_SeguroNovo': np.random.randint(0, 2, 10),
    'md_tipo_renov_ant_bin_3_RenovCongenere': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_01_00_20': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_03_40_60': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_04_60_80': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_05_80_100': np.random.randint(0, 2, 10),
    'MD_tipo_renov_bin_3_RenovPropria_com_sin': np.random.randint(0, 2, 10),
    'idade_ult_RNS_bin_01_05_anos': np.random.randint(0, 2, 10),
    'idade_ult_RNS_bin_06_inf': np.random.randint(0, 2, 10) ,
    'LB': np.random.uniform(100, 200, 10),
    'UB': np.random.uniform(300, 400, 10),
    'Act_cost': np.random.uniform(50, 150, 10)}


In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Criação de um DataFrame de exemplo
data = {
    'premio': np.random.uniform(200, 350, 10),
    'is_casco_ofer': np.random.randint(0, 2, 10),
    'GBM_mercado': np.random.uniform(0.5, 1.5, 10),
    'Premio_IC_100': np.random.uniform(100, 200, 10),
    'premio_anterior': np.random.uniform(150, 300, 10),
    'MD_tempo_casa_2': np.random.uniform(1, 5, 10),
    'per_com': np.random.uniform(0.1, 1.0, 10),
    'per_com_bin_p_1_10': np.random.randint(0, 2, 10),
    'canal_bin_Corretor_Mais': np.random.randint(0, 2, 10),
    'Cod_End_bin_13509': np.random.randint(0, 2, 10),
    'fator_ajuste_bin_f_101_120': np.random.randint(0, 2, 10),
    'classe_bonus_bin_00': np.random.randint(0, 2, 10),
    'UF_bin_AL_CE_RS': np.random.randint(0, 2, 10),
    'UF_bin_AP_DF_ES_PA_PB_RN_SC': np.random.randint(0, 2, 10),
    'UF_bin_GO_MA_MG': np.random.randint(0, 2, 10),
    'UF_bin_MT_PR': np.random.randint(0, 2, 10),
    'UF_bin_RJ': np.random.randint(0, 2, 10),
    'MD_comb_bin_Gasolina': np.random.randint(0, 2, 10),
    'md_tipo_renov_ant_bin_1_SeguroNovo': np.random.randint(0, 2, 10),
    'md_tipo_renov_ant_bin_3_RenovCongenere': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_01_00_20': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_03_40_60': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_04_60_80': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_05_80_100': np.random.randint(0, 2, 10),
    'MD_tipo_renov_bin_3_RenovPropria_com_sin': np.random.randint(0, 2, 10),
    'idade_ult_RNS_bin_01_05_anos': np.random.randint(0, 2, 10),
    'idade_ult_RNS_bin_06_inf': np.random.randint(0, 2, 10),
    'LB': np.random.uniform(100, 200, 10),
    'UB': np.random.uniform(300, 400, 10),
    'Act_cost': np.random.uniform(50, 150, 10) }

df = pd.DataFrame(data)

# Função de demanda usando Regressão Logística 
def Demanda(P, X):
    return np.exp(1.3356 -0.2576 * np.log(P) + 1.3647 * np.divide(P, X[0])-0.8782 * np.log(np.divide(P, X[1]))
                  -0.795 * np.log(np.divide(P, X[2]))-1.6731 * np.log((np.divide(P, X[3])))
                  + 0.3797 * np.log(X[4] + 1) + 0.0011 * X[5] + 0.1253 * X[6] + 0.1334 * X[7]
                  -0.1382 * X[8] + 0.3919 * X[9]-0.1397 * X[10]-0.2305 * X[11]-0.2832 * X[12]
                  + 0.0024 * X[13]-0.1476 * X[14] + 0.2817 * X[15] + 0.3336 * X[16]-0.4115 * X[17]
                  + 0.0127 * X[18]-0.73 * X[19] + 0.4343 * X[20] + 1.1066 * X[21] + 1.9561 * X[22]
                  -0.0742 * X[23] -0.84590*X[24]- 0.7094 * X[25]) /(1 + np.exp(1.3356 -0.2576 * np.log(P) + 1.3647 * np.divide(P, X[0])-0.8782 * np.log(np.divide(P, X[1]))
                  -0.795 * np.log(np.divide(P, X[2]))-1.6731 * np.log((np.divide(P, X[3])))
                  + 0.3797 * np.log(X[4] + 1) + 0.0011 * X[5] + 0.1253 * X[6] + 0.1334 * X[7]
                  -0.1382 * X[8] + 0.3919 * X[9]-0.1397 * X[10]-0.2305 * X[11]-0.2832 * X[12]
                  + 0.0024 * X[13]-0.1476 * X[14] + 0.2817 * X[15] + 0.3336 * X[16]-0.4115 * X[17]
                  + 0.0127 * X[18]-0.73 * X[19] + 0.4343 * X[20] + 1.1066 * X[21] + 1.9561 * X[22]
                  -0.0742 * X[23] -0.84590*X[24]- 0.7094 * X[25]))

# Função de margem
def margem(P, X, C):
    demanda = Demanda(P, X)
    return -(demanda * (P - C))  # Exemplo de função de custo

# Função objetivo para otimizar P com D fixo 
def objetivo(P, D_target, X, C):
    return -margem(P, X, C)  # Maximizando a margem (por isso o "-")

# Função para garantir que P seja o mais próximo de QP possível 
def proximidade_QP(P, D_target, X):
    return abs(np.sum(Demanda(P, X)) - D_target)

# Definir restrições para não negatividade de P 
constraints = [{'type': 'ineq', 'fun': lambda P: P}]  # P >= 0

# Definir limites para P (LB e UB)
limites = np.array(df[['LB', 'UB']])

# Chute inicial (P inicial)
P_inicial = df['premio']

# Definindo o conjunto de variáveis explicativas X 
X = df.drop(columns=['premio', 'LB', 'UB', 'Act_cost'])

# Custo (Act_cost)
C = df['Act_cost']

# QP estipulado (target)
D_target = 0.35 * df.shape[0]  # Exemplo de meta de demanda

# Otimização para encontrar o P ótimo dado o QP estipulado 
resultado = minimize(objetivo, P_inicial, args=(D_target, X.T, C),
                     method='SLSQP', bounds=limites, constraints=constraints)

# Resultados ótimos
P_opt = resultado.x
df['P_opt'] = P_opt

# Exibir resultados
print(f"D estipulado (target): {D_target}") 
print(f"P ótimo: {P_opt}")


In [ ]:
X = df.drop(columns=['premio', 'LB', 'UB', 'Act_cost'])
X.T

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Criação de um DataFrame de exemplo
data = {
    'premio': np.random.uniform(200, 350, 10),
    'is_casco_ofer': np.random.randint(0, 2, 10),
    'GBM_mercado': np.random.uniform(0.5, 1.5, 10),
    'Premio_IC_100': np.random.uniform(100, 200, 10),
    'premio_anterior': np.random.uniform(150, 300, 10),
    'MD_tempo_casa_2': np.random.uniform(1, 5, 10),
    'per_com': np.random.uniform(0.1, 1.0, 10),
    'per_com_bin_p_1_10': np.random.randint(0, 2, 10),
    'canal_bin_Corretor_Mais': np.random.randint(0, 2, 10),
    'Cod_End_bin_13509': np.random.randint(0, 2, 10),
    'fator_ajuste_bin_f_101_120': np.random.randint(0, 2, 10),
    'classe_bonus_bin_00': np.random.randint(0, 2, 10),
    'UF_bin_AL_CE_RS': np.random.randint(0, 2, 10),
    'UF_bin_AP_DF_ES_PA_PB_RN_SC': np.random.randint(0, 2, 10),
    'UF_bin_GO_MA_MG': np.random.randint(0, 2, 10),
    'UF_bin_MT_PR': np.random.randint(0, 2, 10),
    'UF_bin_RJ': np.random.randint(0, 2, 10),
    'MD_comb_bin_Gasolina': np.random.randint(0, 2, 10),
    'md_tipo_renov_ant_bin_1_SeguroNovo': np.random.randint(0, 2, 10),
    'md_tipo_renov_ant_bin_3_RenovCongenere': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_01_00_20': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_03_40_60': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_04_60_80': np.random.randint(0, 2, 10),
    'fx_ir_12m_bin_05_80_100': np.random.randint(0, 2, 10),
    'MD_tipo_renov_bin_3_RenovPropria_com_sin': np.random.randint(0, 2, 10),
    'idade_ult_RNS_bin_01_05_anos': np.random.randint(0, 2, 10),
    'idade_ult_RNS_bin_06_inf': np.random.randint(0, 2, 10),
    'LB': np.random.uniform(100, 200, 10),
    'UB': np.random.uniform(300, 400, 10),
    'Act_cost': np.random.uniform(50, 150, 10) }

df = pd.DataFrame(data)

# Função de demanda usando Regressão Logística 
def Demanda(P, X):
    demanda = []
    for i in range(len(P)):
        demanda_ind = np.exp(1.3356 - 0.2576 * np.log(P[i])
                             + 1.3647 * np.divide(P[i], X[0][i])
                             - 0.8782 * np.log(np.divide(P[i], X[1][i]))
                             - 0.795 * np.log(np.divide(P[i], X[2][i]))
                             - 1.6731 * np.log((np.divide(P[i], X[3][i])))
                             + 0.3797 * np.log(X[4][i] + 1)
                             + 0.0011 * X[5][i]
                             + 0.1253 * X[6][i]
                             + 0.1334 * X[7][i]
                             - 0.1382 * X[8][i]
                             + 0.3919 * X[9][i]
                             - 0.1397 * X[10][i]
                             - 0.2305 * X[11][i]
                             - 0.2832 * X[12][i]
                             + 0.0024 * X[13][i]
                             - 0.1476 * X[14][i]
                             + 0.2817 * X[15][i]
                             + 0.3336 * X[16][i]
                             - 0.4115 * X[17][i]
                             + 0.0127 * X[18][i]
                             - 0.73 * X[19][i]
                             + 0.4343 * X[20][i]
                             + 1.1066 * X[21][i]
                             + 1.9561 * X[22][i]
                             - 0.0742 * X[23][i]
                             - 0.84590 * X[24][i]
                             - 0.7094 * X[25][i]) / (1 + np.exp(1.3356 - 0.2576 * np.log(P[i])
                                                                + 1.3647 * np.divide(P[i], X[0][i])
                                                                - 0.8782 * np.log(np.divide(P[i], X[1][i]))
                                                                - 0.795 * np.log(np.divide(P[i], X[2][i]))
                                                                - 1.6731 * np.log((np.divide(P[i], X[3][i])))
                                                                + 0.3797 * np.log(X[4][i] + 1)
                                                                + 0.0011 * X[5][i]
                                                                + 0.1253 * X[6][i]
                                                                + 0.1334 * X[7][i]
                                                                - 0.1382 * X[8][i]
                                                                + 0.3919 * X[9][i]
                                                                - 0.1397 * X[10][i]
                                                                - 0.2305 * X[11][i]
                                                                - 0.2832 * X[12][i]
                                                                + 0.0024 * X[13][i]
                                                                - 0.1476 * X[14][i]
                                                                + 0.2817 * X[15][i]
                                                                + 0.3336 * X[16][i]
                                                                - 0.4115 * X[17][i]
                                                                + 0.0127 * X[18][i]
                                                                - 0.73 * X[19][i]
                                                                + 0.4343 * X[20][i]
                                                                + 1.1066 * X[21][i]
                                                                + 1.9561 * X[22][i]
                                                                - 0.0742 * X[23][i]
                                                                - 0.84590 * X[24][i]
                                                                - 0.7094 * X[25][i]))
        demanda.append(demanda_ind)
    return np.array(demanda)

# Função de margem
def margem(P, X, C):
    demanda = Demanda(P, X)
    return -(demanda * (P - C))  # Exemplo de função de custo

# Função objetivo para otimizar P com D fixo 
def objetivo(P, D_target, X, C):
    return -margem(P, X, C)  # Maximizando a margem (por isso o "-")

# Função para garantir que P seja o mais próximo de QP possível 
def proximidade_QP(P, D_target, X):
    return abs(np.sum(Demanda(P, X)) - D_target)

# Definir restrições para não negatividade de P 
constraints = [{'type': 'ineq', 'fun': lambda P: P}]  # P >= 0

# Definir limites
limites = np.array(df[['LB', 'UB']])

# Chute inicial (P inicial)
P_inicial = df['premio']

# Definindo o conjunto de variáveis explicativas X 
X = df.drop(columns=['premio', 'LB', 'UB', 'Act_cost'])

# Custo (Act_cost)
C = df['Act_cost']

# QP estipulado (target)
D_target = 0.35 * df.shape[0]  # Exemplo de meta de demanda

# Otimização para encontrar o P ótimo dado o QP estipulado 
resultado = minimize(objetivo, P_inicial, args=(D_target, X.T, C),
                     method='SLSQP', bounds=limites, constraints=constraints)

# Resultados ótimos
P_opt = resultado.x
df['P_opt'] = P_opt

# Exibir resultados
print(f"D estipulado (target): {D_target}") 
print(f"P ótimo: {P_opt}")



In [ ]:
import autograd
from autograd import grad

In [ ]:
import autograd.numpy as np
from autograd import grad

def f(P, X):
    return np.exp(0.64 + 0.4 * X - 0.005 * P) / (1 + np.exp(0.64 + 0.4 * X - 0.005 * P))

f_prime = grad(f)

P = 287.5960434220056
X = 1
print(f_prime(P, X))
 

In [ ]:
P2=257.7833159899672
X = 0
print(f_prime(P, X))

In [ ]:
f_prime

In [ ]:
!pip install folium


In [ ]:
!pip install geopandas

In [ ]:
P2: 257.7833159899672

In [ ]:
# Dados fornecidos

dados = df_lote_otim[[
    'is_casco_ofer',
    'GBM_mercado',
    'Premio_IC_100',
    'premio_anterior',
    'MD_tempo_casa_2',
    'per_com',
    'per_com_bin_p_1_10',
    'canal_bin_Corretor_Mais',
    'Cod_End_bin_13509',
    'fator_ajuste_bin_f_101_120',
    'classe_bonus_bin_00',
    'UF_bin_AL_CE_RS',
    'UF_bin_AP_DF_ES_PA_PB_RN_SC',
    'UF_bin_GO_MA_MG',
    'UF_bin_MT_PR',
    'UF_bin_RJ',
    'MD_comb_bin_Gasolina',
    'md_tipo_renov_ant_bin_1_SeguroNovo',
    'md_tipo_renov_ant_bin_3_RenovCongenere',
    'fx_ir_12m_bin_01_00_20',
    'fx_ir_12m_bin_03_40_60',
    'fx_ir_12m_bin_04_60_80',
    'fx_ir_12m_bin_05_80_100',
    'MD_tipo_renov_bin_3_RenovPropria_com_sin',
    'idade_ult_RNS_bin_01_05_anos',
    'idade_ult_RNS_bin_06_inf'
]]



df = dados.iloc[0:100,]

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Dados fornecidos

dados = df_lote_otim[[
    'premio',
    'is_casco_ofer',
    'GBM_mercado',
    'Premio_IC_100',
    'premio_anterior',
    'MD_tempo_casa_2',
    'per_com',
    'per_com_bin_p_1_10',
    'canal_bin_Corretor_Mais',
    'Cod_End_bin_13509',
    'fator_ajuste_bin_f_101_120',
    'classe_bonus_bin_00',
    'UF_bin_AL_CE_RS',
    'UF_bin_AP_DF_ES_PA_PB_RN_SC',
    'UF_bin_GO_MA_MG',
    'UF_bin_MT_PR',
    'UF_bin_RJ',
    'MD_comb_bin_Gasolina',
    'md_tipo_renov_ant_bin_1_SeguroNovo',
    'md_tipo_renov_ant_bin_3_RenovCongenere',
    'fx_ir_12m_bin_01_00_20',
    'fx_ir_12m_bin_03_40_60',
    'fx_ir_12m_bin_04_60_80',
    'fx_ir_12m_bin_05_80_100',
    'MD_tipo_renov_bin_3_RenovPropria_com_sin',
    'idade_ult_RNS_bin_01_05_anos',
    'idade_ult_RNS_bin_06_inf'
]]



#df = dados.iloc[0:100,]
df = dados

# Definindo a função de demanda
def demanda(P, *X):
    D = np.exp(1.3356 - 0.2576 * np.log(P) + 1.3647 * (P / X[0]) - 0.8782 * np.log(P / X[1]) -
               0.795 * np.log(P / X[2]) - 1.6731 * np.log(P / X[3]) + 0.3797 * np.log(X[4] + 1) +
               0.0011 * X[5] + 0.1253 * X[6] + 0.1334 * X[7] - 0.1382 * X[8] + 0.3919 * X[9] -
               0.1397 * X[10] - 0.2305 * X[11] - 0.2832 * X[12] + 0.0024 * X[13] - 0.1476 * X[14] +
               0.2817 * X[15] + 0.3336 * X[16] - 0.4115 * X[17] + 0.0127 * X[18] - 0.73 * X[19] +
               0.4343 * X[20] + 1.1066 * X[21] + 1.9561 * X[22] - 0.0742 * X[23] - 0.8459 * X[24] -
               0.7094 * X[25]) / (1 + np.exp(1.3356 - 0.2576 * np.log(P) + 1.3647 * (P / X[0]) - 0.8782 * np.log(P / X[1]) -
               0.795 * np.log(P / X[2]) - 1.6731 * np.log(P / X[3]) + 0.3797 * np.log(X[4] + 1) +
               0.0011 * X[5] + 0.1253 * X[6] + 0.1334 * X[7] - 0.1382 * X[8] + 0.3919 * X[9] -
               0.1397 * X[10] - 0.2305 * X[11] - 0.2832 * X[12] + 0.0024 * X[13] - 0.1476 * X[14] +
               0.2817 * X[15] + 0.3336 * X[16] - 0.4115 * X[17] + 0.0127 * X[18] - 0.73 * X[19] +
               0.4343 * X[20] + 1.1066 * X[21] + 1.9561 * X[22] - 0.0742 * X[23] - 0.8459 * X[24] -
               0.7094 * X[25]))
    return D 

# Função objetivo para minimizar a diferença entre a demanda total e a demanda alvo
def objetivo(P, X_val, D_alvo):
    demanda_total = sum([demanda(P[i], *X_val[i]) for i in range(len(P))])
    return (demanda_total - D_alvo)**2

# Definindo a demanda alvo
D_alvo = 0.65

# Valores iniciais para P
P_inicial = df['premio'].values

# Limites para P
limites = [(0.9 * p, 1.1 * p) for p in P_inicial]

# Função objetivo compatível com o otimizador
def objetivo_np(P):
    X_val = df.drop(columns=['premio']).values
    return objetivo(P, X_val, D_alvo)

# Otimização para encontrar P_otimo
resultado = minimize(objetivo_np, P_inicial, bounds=limites)

# Resultados
P_otimo = resultado.x

print(f"P_otimo: {P_otimo}")

# Verificando a demanda com os preços otimizados
X_val = df.drop(columns=['premio']).values
demanda_total = sum([demanda(P_otimo[i], *X_val[i]) for i in range(len(P_otimo))])

print(f"Demanda total: {demanda_total}")
print(f"Demanda alvo (D_alvo): {D_alvo}")

In [ ]:
P_otimo.mean()

In [ ]:
di = [demanda(P_otimo[i], *X_val[i]) for i in range(len(P_otimo))]
di

In [ ]:
np.mean(di)